In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import ChatHuggingFace, HuggingFaceEmbeddings, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.documents import Document

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [2]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]



In [3]:
embedding_model= HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

vectorstore= FAISS.from_documents(
    documents=docs,
    embedding= embedding_model,
)

base_retriever= vectorstore.as_retriever(search_kwargs={"k":5})

d:\Users\Aditya_Kumar\LangChainCampusX\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
load_dotenv()
# set up the compressor using on LLM

llm= HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-1.5B-Instruct",
    task="text-generation",
    huggingfacehub_api_token= os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

model= ChatHuggingFace(llm= llm)

compressor= LLMChainExtractor.from_llm(model)

In [13]:
# create the contextual compression retriever
compression_retriever= ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor= compressor,
)

In [11]:
# query the retriever
query = "What is photosynthesis?"

compressed_results= compression_retriever.invoke(query)

In [12]:
for i, doc in enumerate(compressed_results):
    print(f"\n---Result {i+1}---")
    print(f"{doc.page_content}")


---Result 1---
Photosynthesis is the process by which green plants convert sunlight into energy.


[]
